In [1]:
# Hidden cell for imports
import pychor
import galois

# Oblivious Transfer

Oblivious Transfer (OT) is a fundamental primitive used in many MPC protocols. As we'll see at the end of this chapter, it can be used to build a protocol for generating multiplication triples. It is also used as part of the online portion of other MPC protocols, as we'll see in later chapters.

For more information on Oblivious Transfer, see **Sections 2.4 and 3.7 of [Pragmatic MPC](https://securecomputation.org/)**.

## The Oblivious Transfer Functionality

OT is a protocol between two parties: the *Sender* and the *Receiver*. The Sender provides two inputs, and the Receiver selects one of them. At the end of the protocol, the Receiver gets *only* one of the Sender's inputs (the selected one), and the Sender *does not know which input was selected* (this is the "oblivious" part - the Sender is oblivious to the selection decision). In fact, the Sender does not receive any output from OT. The ideal functionality for this version of OT, which is called 1-out-of-2 OT because the sender provides two inputs, is defined as follows:

````{admonition} Ideal functionality: 1-out-of-2 Oblivious Transfer
:class: tip

1. The Sender sends the secret inputs $x_0$ and $x_1$ to $\mathcal{F}_\text{OT}$
2. The Receiver sends the selection bit $s$ to $\mathcal{F}_\text{OT}$
3. If $s = 0$, $\mathcal{F}_\text{OT}$ sends $x_0$ to the Receiver; if $s = 1$, $\mathcal{F}_\text{OT}$ sends $x_1$ to the Receiver
````

We can implement the functionality as follows. Since OT usually operates over bits, we'll use values in $GF(2)$ (i.e. just the bits 0 and 1).

In [2]:
sender = pychor.Party('sender')
receiver = pychor.Party('receiver')
FOT = pychor.Party('FOT')

@pychor.local_function
def select(inputs, selection):
    return inputs[selection]

def functionality_ot2(sender, receiver, inputs, selection):
    inputs.send(sender, FOT)
    selection.send(receiver, FOT)
    selected_input = select(inputs, selection)
    selected_input.send(FOT, receiver)
    return selected_input

In [3]:
with pychor.LocalBackend():
    inputs = sender.constant([42, 67])

    selection = receiver.constant(0)
    selected_input = functionality_ot2(sender, receiver, inputs, selection)
    print('OT Result (s=0):', selected_input)

    selection = receiver.constant(1)
    selected_input = functionality_ot2(sender, receiver, inputs, selection)
    print('OT Result (s=1):', selected_input)

OT Result (s=0): 42@{FOT, receiver}
OT Result (s=1): 67@{FOT, receiver}


## A Protocol for Oblivious Transfer

In this section we'll define a simple protocol for OT that realizes the functionality in the semi-honest setting. Our approach follows Section 3.7 of [Pragmatic MPC](https://securecomputation.org/) and uses [public-key cryptography](https://en.wikipedia.org/wiki/Public-key_cryptography) as a fundamental building block. The asymmetry between Sender and Receiver in OT is very different from what we've seen in prior chapters, and it turns out that this asymmetry creates a deep link between OT and public-key cryptography (see Gertner et al. {cite}`gertner2000relationship`) for details).

We'll use the [PyNaCl](https://pypi.org/project/PyNaCl/) library for public-key cryptography. This library is a wrapper around [libsodium](https://github.com/jedisct1/libsodium), a high-performance cryptography library. The [documentation for public-key cryptography in PyNaCl can be found here](https://pynacl.readthedocs.io/en/latest/public/). Here's a simple example that generates a key pair, encrypts a value, and then decrypts the value. When using the PyNaCl library, everything is bytes (input values, keys, encrypted values, etc) - so we will often need to encode the values we want to encrypt into bytestrings before doing so.

In [4]:
from nacl.public import PrivateKey, PublicKey, SealedBox

# Generate the key pair (secret key and public key)
key_pair = PrivateKey.generate()
print('Key pair:', key_pair)

# Encrypt a value using the public key from the key pair
pk = key_pair.public_key
encrypted_value = SealedBox(pk).encrypt(b'this is a test')
print('Encrypted value:', encrypted_value)

# Decrypt the value using the secret key from the key pair
decrypted_value = SealedBox(key_pair).decrypt(encrypted_value)
print('Decrypted value:', decrypted_value)

Key pair: b'\x87T\xb8+l\xb0\t\x15\x17Ntc\xb6\n\x1e\xc4\xdd*\x1c3\xa1\xf2\x03"H\x13\xf6\x96\x96\x19.\xb8'
Encrypted value: b"\xed\xd2\xbc\xd2\xff\xc8e;\x9c>(\xe7Y\xc7\xeaE\xe2\n\x9fD\xdb\xe5\x03\xb8Sq\xf7\xb4\x92\xed\x0ba'<[B\x0f\xccR\x1d\xbbXN\x9aK\x83#\xdc3m\xa8E\x19c\xc3\xb1\x91\x85\xdf\x8dMW"
Decrypted value: b'this is a test'


In our OT protocol, the basic idea is for the Receiver to send two public keys to the Sender, and for the Sender to encrypt *both* secret inputs (one input per public key) and send the encrypted inputs back to the Receiver. However, the two public keys are special: one is generated as part of a key pair (so the Receiver knows the corresponding secret key), and the other is *totally random* (so no secret key exists, and *the Receiver cannot decrypt any message encrypted with it*). Which one is "real" and which one is random depends on the selection bit. This approach satisfies the requirements of the ideal functionality: the Receiver gets only the selected input (because they can't decrypt the other one), and the Sender gets nothing (because the two public keys both look random, without knowledge of how they were generated).

Note that this protocol is *very* insecure in the presence of malicious adversaries. For example, a malicious Receiver could send two "real" public keys to the Sender, and then trivially decrypt both inputs.

````{admonition} Protocol: 1-out-of-2 Oblivious Transfer
:class: note

Setup:
- Sender has secret inputs $x_0$ and $x_1$
- Receiver has selection bit $s$

The parties Sender and Receiver follow the following steps:
1. The Receiver generates a key pair $sk$, $pk$, and samples a random public key, $pk'$, from the public key space. If $s = 0$, the Receiver sends the pair $(pk, pk')$ to the Sender; if $s = 1$, the receiver sends the pair $(pk', pk)$ to the Sender.
2. The Sender receives $(pk_0, pk_1)$ and sends back to the Receiver two encryptions $e_0 = \text{Enc}_{pk_0}(x_0), e_1 = \text{Enc}_{pk_1}(x_1)$.
3. The Receiver receives $e_0, e_1$ and decrypts the ciphertext $e_s$ using $sk$. The Receiver is unable to decrypt the second ciphertext as it does not have the corresponding secret key.
````

The implementation follows the same steps, but uses the PyNaCl API to perform the key generation, encryption, and decryption operations. Since PyNaCl only supports encrypting bytes, the protocol converts the secret inputs to bytestrings (the protocol expects the inputs to be integers, and returns an integer).

In [14]:
from nacl.utils import random

@pychor.local_function
def encrypt_inputs(pub_keys, inputs):
    # Encode the inputs as bytes
    length = max([(int(x).bit_length() + 7) // 8 for x in inputs])
    inputs_bytes = [int(x).to_bytes(length, 'little') for x in inputs]

    # Encrypt the inputs
    encrypted_inputs = [SealedBox(pk).encrypt(x) for pk, x in \
                        zip(pub_keys, inputs_bytes)]
    return encrypted_inputs

@pychor.local_function
def decrypt_result(selection, key, encrypted_inputs):
    # Select the correct input
    selected_input = encrypted_inputs[selection]
    # Decrypt it and convert it from bytes to int
    plaintext = SealedBox(key).decrypt(selected_input)
    return int.from_bytes(plaintext, 'little')

def protocol_ot2(sender, receiver, inputs, selection):
    @pychor.local_function
    def gen_keys(selection):
        # Generate sk, pk
        key = PrivateKey.generate()
        fake_key = PublicKey(random(PublicKey.SIZE))
        if selection == 0:
            return key, (key.public_key, fake_key)
        else:
            return key, (fake_key, key.public_key)

    # Step 1: Generate keys and send to Sender
    sk, pub_keys = gen_keys(selection).untup(2)
    pub_keys.send(receiver, sender)

    # Step 2: Encrypt inputs and send to Receiver
    encrypted_inputs = encrypt_inputs(pub_keys, inputs)
    encrypted_inputs.send(sender, receiver)

    # Step 3: Decrypt result
    result = decrypt_result(selection, sk, encrypted_inputs)

    return result

In [15]:
with pychor.LocalBackend():
    inputs = sender.constant([42, 67])

    selection = receiver.constant(0)
    selected_input = protocol_ot2(sender, receiver, inputs, selection)
    print('OT Result (s=0):', selected_input)

    selection = receiver.constant(1)
    selected_input = protocol_ot2(sender, receiver, inputs, selection)
    print('OT Result (s=1):', selected_input)

OT Result (s=0): 42@{receiver}
OT Result (s=1): 67@{receiver}


## Generating Multiplication Triples using OT

One important use of OT is to generate multiplication triples. We previously introduced the ideal functionality for generating multiplication triples, but we didn't implement a protocol realizing it because we didn't have the necessary tools. With OT, now we do!

We'll start with the case of generating *binary* multiplication triples: the case where all secret values and secret shares are in $GF(2)$ (i.e. either 0 or 1). Since $GF(2)$ is a finite field, the protocols we built earlier that use multiplication triples will work just fine in this case.

Recall that our goal is to generate secret-shared $a$, $b$, and $c$ such that $a*b = c$. Specifically, we want to end up in the following situation:
- P1 knows $a_1$, $b_1$, $c_1$
- P2 knows $a_2$, $b_2$, $c_2$
- $(a_1+a_2)(b_1 + b_2) = c_1 + c_2$

To get there, we'll use this basic idea:
1. P1 picks $a_1$, $b_1$, $c_1$ randomly
2. P2 picks $a_2$, $b_2$ randomly
3. P1 and P2 run OT, with P1 as sender and P2 as receiver, so that P2 ends up with $c_2$

What should the secret inputs and selection bit for OT be? We actually need *two* selection bits, and they'll be the values of P2's shares ($a_2$ and $b_2$). For the inputs, since we're working on $GF(2)$, P1 can build the a big truth table to list the possible values of $c_2$, and use the truth table to generate the set of secret inputs. An excerpt of the complete truth table for valid multiplication triples looks like this:

| $a_1$ | $b_1$ | $c_1$ | $a_2$ | $b_2$ | $c_2$ |
|-------|-------|-------|-------|-------|-------|
| 0     | 0     | 0     | 0     | 0     | 0     |
| 0     | 0     | 0     | 0     | 1     | 0     |
| 0     | 0     | 0     | 1     | 0     | 0     |
| 0     | 0     | 0     | 1     | 1     | 1     |
...

The complete table is large, so we don't show it here. But to generate multiplication triples, we won't actually have to build the whole thing. When P1 builds the truth table to use in OT, P1 actually knows many of the values of the secret shares listed in it, and so most of the rows in the table can be eliminated. The only values P1 *doesn't* know are $a_2$ and $b_2$, and there are only four possible combinations of those values. So, we can write a function that builds the truth table for $a_2$ and $b_2$, given the values of the other shares:

In [13]:
GF_2 = galois.GF(2)

def truth_table(a1, b1, c1):
    possible_c2s = []
    # Consider all possibilities for a2 and b2
    for a2 in GF_2([0, 1]):
        for b2 in GF_2([0, 1]):
            # Compute the share c2 in terms of the others
            c2 = (a1+a2) * (b1+b2) - c1
            possible_c2s.append(c2)
    return possible_c2s

truth_table(GF_2(0), GF_2(0), GF_2(0))

[GF(0, order=2), GF(0, order=2), GF(0, order=2), GF(1, order=2)]

Note that these values are the first four values of $c_2$ in the truth table excerpt above.

The other thing we'll need is a fancier version of OT itself. We've seen an OT protocol with two secret inputs and one selection bit, but our situation calls for a version with *four* secret inputs (the four values in the output above) and *two* selection bits (the shares $a_2$ and $b_2$). This is called 1-out-of-4 OT, and we'll build this protocol shortly.

The complete protocol for generating a binary multiplication triple can be defined as follows:

````{admonition} Protocol: Generate Binary Multiplication Triple
:class: note

The parties P1 and P2 follow the following steps:
1. P1 picks $a_1$, $b_1$, and $c_1 \in GF(2)$ randomly
2. P2 picks $a_2$, $b_2 \in GF(2)$ randomly
3. P1 generates the truth table based on its shares by running `truth_table`
4. P1 and P2 run 1-out-of-4 OT, with P1 as sender and P2 as receiver. P1 submits the truth table as the set of secret inputs, and P2 submits the values of $a_2$ and $b_2$ as the selection bits. P2 receives the output of OT, which is the value of $c_2$.

At the end of the protocol, P1 and P2 hold secret shares of $a$, $b$, and $c$, such that $a$ and $b$ are random and unknown to either party, and $a*b = c$.
````

### 1-out-of-4 OT

First, we adapt the earlier 1-out-of-2 OT protocol to the 1-out-of-4 case:

````{admonition} Protocol: 1-out-of-4 Oblivious Transfer
:class: note

Setup:
- Sender has secret inputs $x_0, x_1, x_2, x_3$
- Receiver has selection index $s \in \{0, 1, 2, 3\}$

The parties Sender and Receiver follow the following steps:
1. The Receiver generates a key pair $sk$, $pk$, and samples 3 random public keys from the public key space. The Receiver sends the tuple $(pk_0, pk_1, pk_2, pk_3)$ to the Sender, where $pk_s = pk$ and $pk_i$ where $i \not = s$ is one of the random keys.
2. The Sender receives $(pk_0, pk_1, pk_2, pk_3)$ and sends back to the Receiver four encryptions $(e_0, e_1, e_2, e_3)$ where $e_i = \text{Enc}_{pk_i}(x_i)$.
3. The Receiver receives $(e_0, e_1, e_2, e_3)$ and decrypts the ciphertext $e_s$ using $sk$. The Receiver is unable to decrypt the other ciphertexts.
````

It's simple to generalize the idea behind OT to any number of secret inputs in this way, and it just requires increasing the number of selection bits correspondingly. We can implement this protocol in a very similar way to 1-out-of-2 OT, repeating much of the code:

In [16]:
def protocol_ot4(sender, receiver, inputs, selection):
    @pychor.local_function
    def gen_keys(selection):
        # Generate sk, pk
        key = PrivateKey.generate()
        public_keys = [PublicKey(random(PublicKey.SIZE)) for _ in range(4)]
        public_keys[selection] = key.public_key
        return key, public_keys

    # Step 1: Generate keys and send to Sender
    sk, pub_keys = gen_keys(selection).untup(2)
    pub_keys.send(receiver, sender)

    # Step 2: Encrypt inputs and send to Receiver
    encrypted_inputs = encrypt_inputs(pub_keys, inputs)
    encrypted_inputs.send(sender, receiver)

    # Step 3: Decrypt result
    result = decrypt_result(selection, sk, encrypted_inputs)

    return result

In [17]:
with pychor.LocalBackend():
    inputs = sender.constant([42, 67, 95, 87])

    for i in range(4):
        selection = receiver.constant(i)
        selected_input = protocol_ot4(sender, receiver, inputs, selection)
        print(f'OT Result (s={i}):', selected_input)

OT Result (s=0): 42@{receiver}
OT Result (s=1): 67@{receiver}
OT Result (s=2): 95@{receiver}
OT Result (s=3): 87@{receiver}
